# Project to automate api access 
+ I added my api key as config.py file  PLease get your api key https://newsapi.org/docs/get-started 

In [1]:
import pandas as pd
import config
import math
from newsapi import NewsApiClient
import newspaper
import requests
from newspaper import fulltext
import time
# Hit Api with credentials
newsapi = NewsApiClient(api_key=config.api_key)


### Grab all sources
+ read through available sources list and make df storing domain and source name
+ I hit a ton of sources below, we can clearly narrow it down 

### Choosing data sources
+ Lets attempt to grab some sources from different geographic locations as well as different idological perspectives

+ categorizing news sources
    + Traditional TV MSM
        +  http://us.cnn.com   
        +  http://www.cnbc.com 
        +  http://www.foxnews.com  
        +  http://www.msnbc.com  
        +  https://abcnews.go.com  
        +  http://www.nbcnews.com  
    + Traditional publications 
        +  http://www.nytimes.com  
        +  https://www.washingtonpost.com 
        
    + Internet Sources
        +  http://www.huffingtonpost.com 
        +  https://www.politico.com
        +  http://www.breitbart.com 
        +  https://news.google.com 
        +  https://www.buzzfeed.com 
        +  https://news.vice.com  
    + Financial publications
        +  http://www.economist.com
        +  http://www.bloomberg.com 
        +  http://www.businessinsider.com 
        +  http://www.wsj.com
        +  http://fortune.com  
        
    + News aggregators
        +  https://apnews.com/ 
        +  http://www.reuters.com 
    + foreign reporting
         + http://www.aljazeera.com  
         + http://www.bbc.co.uk/news   
         + https://www.jpost.com/  
         + http://timesofindia.indiatimes.com 
         + https://russian.rt.com 
         + https://www.theguardian.com/uk 
         + http://www.independent.co.uk  
         + http://www.telegraph.co.uk  


 
### Build Query String 

In [17]:
url = 'https://raw.githubusercontent.com/mburke65/Data_698_Final/master/lists.csv'
df = pd.read_csv(url, error_bad_lines=False)


## Literally picking data sources from df i printed above 
a= df.iloc[[16,17,24],:]
b= df.iloc[[40,43,77],:]
list_sources =a["sources"].tolist()

## build out string for query request 
myString = ",".join(list_sources)

list_sources2 = b["sources"].tolist()
myString2 = ",".join(list_sources2)
myString2

'fox-news,google-news,msnbc'

## Now lets begin process of automating query calls
+ first lets build function to clean query returns

In [7]:
def clean_query(query):
    for x in query['articles']:
        try:
            x["source"] = x["source"]["name"]
        except:
            pass
        try:
            x['publishedAt'] = str.split(x['publishedAt'], "T")[0]
        except:
            pass
        try:
            del x['urlToImage']
        except KeyError:
            pass
    my_df = pd.DataFrame(query["articles"])
    return my_df

## Function to hit the api
+ Originally I had a loop here.  instead I figured I would just build a function that takes start data, end data, query term(candidate)
    + The original code kept giving me a query limit reached result, so I decided to change up strategy and search one day at a time
    + After we hit the papers at the start for past 30 days, we will only need 1 day at a time going forward.
    + I built in some print statements for error handeling, which you will see below in the block after this

In [8]:
import sys
import time
candidates_list=[]
total_count=[]

# Make first call
def hit_api(start,end,q,myString):
    
    ## catch bug with formatted strings for dates
    if end < 10:
        start_str = "0"+ str(start)
        end_str = "0"+ str(end)
    elif end==10:
        start_str = "0"+ str(start)
        end_str = str(end)
    else :
        start_str = str(start)
        end_str = str(end)
      
    ## API query
    all_articles = newsapi.get_everything(q=q,
                                          sources=myString,
                                          language='en',
                                          from_param='2019-09-{}'.format(start_str),
                                          to='2019-09-{}'.format(end_str),
                                          sort_by='relevancy',
                                          page_size=100,
                                          page=1)
    
    ## get count
    total_pages = math.ceil(all_articles["totalResults"]/100)
    print("query will return: "+ str(all_articles["totalResults"]))
    
    ## store count to check versus dimension of df later
    total_count.append(all_articles["totalResults"])
    
    ## Clen query 
    all_articles = clean_query(all_articles)
    
    ## append to list
    candidates_list.append(all_articles)
    return(candidates_list)


## Built out a loop 
+ Simple, look at first day of September to last day incrementing start and end by 1 each time



In [9]:
for x in range(20,31):
    df=hit_api(x,x+1,'Biden OR Joe Biden',myString)
    
## collapse list on itself to build big df
Bernie_df = pd.concat(df)
Bernie_df= Bernie_df.reset_index(drop=True)

for x in range(20, 31):
    df = hit_api(x, x+1, 'Biden OR Joe Biden', myString)

# collapse list on itself to build big df
Cand_df = pd.concat(df)
Cand_df = Cand_df.reset_index(drop=True)
error_checks()


## Build counts as check on progress of loop. will count down Url left to hit
counts=Cand_df['url'].shape[0]
list_full_text=[]
for link in Cand_df['url']:
    counts-=1
    print(counts)
    html = requests.get(link,headers=headers).text
    try: text = fulltext(html)
    except: 
        print("no words found")
        text="no words found"
        list_full_text.append(text)
        continue
    list_full_text.append(text)
Cand_df['full_art']=pd.Series(list_full_text)
Cand_df.to_csv('data_csvs/data1'+str(time.time())+'.csv')

query will return: 0
query will return: 2
query will return: 4
query will return: 3
query will return: 3
query will return: 3
query will return: 2
query will return: 2
query will return: 1
query will return: 3
query will return: 39


## Data sanity checks

### Issues
+ The counts don't match up between expected and actual because the query results are limited to first 100 hits
    + Won't be an issue with less publications, but we need to be careful splitting candidate calls up.
+ Not all links are unique
    + This is expected given internet sources reposting articles from other sources

In [10]:
def error_checks():
    ##validate that df shape[0] is equal to expected query count
    print("we should expect: {} articles".format(sum(total_count)))
    print("we have: {} articles".format(Bernie_df.shape[0]))


    ## test 100 results theory
    over_100 = sum([x-100 for x in total_count if x>100 ])
    print("{} results came into query that exceeded 100 hits in a day.".format(over_100))


    ##unique links
    unique_array = Bernie_df.url.unique()
    print("we have {} unique links".format(unique_array.shape[0]))
    
    duplicateRowsDF = Bernie_df[Bernie_df.duplicated("url")]
    print("we have duplciate shape df of: {} ".format(duplicateRowsDF.shape))


error_checks()

we should expect: 62 articles
we have: 62 articles
0 results came into query that exceeded 100 hits in a day.
we have 49 unique links
we have duplciate shape df of: (13, 7) 


### Hitting newspaper 3k Api with links
+ Count print showing amount articles left
+ No words found prints it article has no words
+ headers needs to be custom changed to your own headers

In [11]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'}
## Build counts as check on progress of loop. will count down Url left to hit
counts=Bernie_df['url'].shape[0]
list_full_text=[]
for link in Bernie_df['url']:
    counts-=1
    print(counts)
    html = requests.get(link,headers=headers).text
    try: text = fulltext(html)
    except: 
        print("no words found")
        text="no words found"
        list_full_text.append(text)
        continue
    list_full_text.append(text)

61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0


Bernie_df["url"]

## Add text to previous dataframe

In [12]:
Bernie_df['full_art']=pd.Series(list_full_text)

In [15]:
Bernie_df['full_art'][1]

'Why did this happen?\n\nPlease make sure your browser supports JavaScript and cookies and that you are not blocking them from loading. For more information you can review our Terms of Service and Cookie Policy.'

In [ ]:
import time
Bernie_df.to_csv('Bloomberg'+str(time.time())+'.csv')

## Conclusion
+ we need to functionize streamline and clean up query calls.
+ sorry my python is rusty.

In [ ]:
time.time()

In [ ]:
Bernie_df

In [ ]:
aa=pd.read_csv("data_1571550604.259001.csv")

In [ ]:
aa.shape

In [ ]:
aa


In [ ]:
aa.loc[119,'full_art']